In [2]:
import os
import requests
import zipfile

# Retrieve the data
if not os.path.exists(os.path.join('data','tiny-imagenet-200.zip')):
    url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
    # Get the file from web
    r = requests.get(url)

    if not os.path.exists('data'):
        os.mkdir('data')
    
    # Write to a file
    with open(os.path.join('data','tiny-imagenet-200.zip'), 'wb') as f:
        f.write(r.content)
else:
    print("The zip file already exists.")
    
if not os.path.exists(os.path.join('data', 'tiny-imagenet-200')):
    with zipfile.ZipFile(os.path.join('data','tiny-imagenet-200.zip'), 'r') as zip_ref:
        zip_ref.extractall('data')
else:
    print("The extracted data already exists")

The zip file already exists.
The extracted data already exists


In [6]:
from functools import partial
import tensorflow as tf
#import tensorflow_hub as hub
import requests
import zipfile
import requests
import os
import time
import pandas as pd
import random
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, AvgPool2D, Dense, Concatenate, Flatten, Lambda, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.losses import CategoricalCrossentropy
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger
import numpy as np
from PIL import Image
import tensorflow.keras.backend as K
import pickle
from tensorflow.keras.models import load_model, Model
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except:
        print("Couldn't set memory_growth")
        pass
    
    
def fix_random_seed(seed):
    """ Setting the random seed of various libraries """
    try:
        np.random.seed(seed)
    except NameError:
        print("Warning: Numpy is not imported. Setting the seed for Numpy failed.")
    try:
        tf.random.set_seed(seed)
    except NameError:
        print("Warning: TensorFlow is not imported. Setting the seed for TensorFlow failed.")
    try:
        random.seed(seed)
    except NameError:
        print("Warning: random module is not imported. Setting the seed for random failed.")

# Fixing the random seed
random_seed = 1997
fix_random_seed(random_seed)

print("TensorFlow version: {}".format(tf.__version__))

TensorFlow version: 2.10.0


In [5]:
#pip install tensorflow==2.10.0

  Using cached tensorflow-2.10.0-cp310-cp310-win_amd64.whl.metadata (3.1 kB)
  Using cached protobuf-3.19.6-cp310-cp310-win_amd64.whl.metadata (806 bytes)
  Using cached tensorboard-2.10.1-py3-none-any.whl.metadata (1.9 kB)
  Using cached keras-2.10.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl.metadata (1.1 kB)
Using cached tensorflow-2.10.0-cp310-cp310-win_amd64.whl (455.9 MB)
Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
Using cached protobuf-3.19.6-cp310-cp310-win_amd64.whl (895 kB)
Using cached tensorboard-2.10.1-py3-none-any.whl (5.9 MB)
Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Attempting uninstall: keras
    Found existing installation: keras 3.1.1
    Uninstalling keras-3.1.1:
      Successfully uninstalled keras-3.1.1
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      S

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-datasets 4.9.4 requires tqdm, which is not installed.
tensorflow-datasets 4.9.4 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.
tf-keras 2.16.0 requires tensorflow<2.17,>=2.16, but you have tensorflow 2.10.0 which is incompatible.


In [11]:
def get_test_labels_df(test_labels_path):
    """ Reading the test data labels for all files in the test set as a data frame """
    test_df = pd.read_csv(test_labels_path, sep='\t', index_col=None, header=None)
    test_df = test_df.iloc[:,[0,1]].rename({0:"filename", 1:"class"}, axis=1)
    return test_df

def get_train_valid_test_data_generators(batch_size, target_size):
    """ Get the training/validation/testing data generators """
    
    # Defining a data-augmenting image data generator and a standard image data generator
    image_gen_aug = ImageDataGenerator(
        samplewise_center=False, rotation_range=30, width_shift_range=0.2,
        height_shift_range=0.2, brightness_range=(0.5,1.5), shear_range=5, 
        zoom_range=0.2, horizontal_flip=True, fill_mode='constant', cval=127.5, 
        validation_split=0.1
    )
    image_gen = ImageDataGenerator(samplewise_center=False)
    
    # Define a training data generator
    partial_flow_func = partial(
        image_gen_aug.flow_from_directory, 
        directory=os.path.join('data','tiny-imagenet-200', 'train'), 
        target_size=target_size, classes=None,
        class_mode='categorical', batch_size=batch_size, 
        shuffle=True, seed=random_seed)
    
    # Get the training data subset
    train_gen = partial_flow_func(subset='training')
    # Get the validation data subset
    valid_gen = partial_flow_func(subset='validation')    

    # Defining the test data generator
    test_df = get_test_labels_df(os.path.join('data','tiny-imagenet-200',  'val', 'val_annotations.txt'))
    test_gen = image_gen.flow_from_dataframe(
        test_df, directory=os.path.join('data','tiny-imagenet-200',  'val', 'images'), target_size=target_size, classes=None,
        class_mode='categorical', batch_size=batch_size, shuffle=False
    )
    return train_gen, valid_gen, test_gen


batch_size = 48
target_size = (224,224)
# Getting the train,valid, test data generators
train_gen, valid_gen, test_gen = get_train_valid_test_data_generators(batch_size, target_size)
# Modifying the data generators to fit the model targets
train_gen_inceptionV3, valid_gen_inceptionV3, test_gen_inceptionV3 = get_train_valid_test_data_generators(batch_size,(299,299))

with open(os.path.join('data','class_indices'), 'wb') as f:
    pickle.dump(train_gen.class_indices, f)

Found 90000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.
Found 10000 validated image filenames belonging to 200 classes.
Found 90000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.
Found 10000 validated image filenames belonging to 200 classes.


In [12]:
from tensorflow.keras.applications import VGG19,ResNet50V2,InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
def create_inceptionv3_model(input_shape=(299, 299, 3), num_classes=200):
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
    x = GlobalAveragePooling2D()(base_model.output)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model

inceptionv3_model = create_inceptionv3_model()
inceptionv3_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

def create_resnet50v2_model(input_shape=(224, 224, 3), num_classes=200):
    base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)
    x = GlobalAveragePooling2D()(base_model.output)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model

resnet50v2_model = create_resnet50v2_model()
resnet50v2_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
def create_vgg19_model(input_shape=(224, 224, 3), num_classes=200):
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
    x = Flatten()(base_model.output)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model

vgg19_model = create_vgg19_model()
vgg19_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#vgg19_model.summary()
#resnet50v2_model.summary()
#inceptionv3_model.summary()

In [13]:
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger, ReduceLROnPlateau
es_callback = EarlyStopping(monitor='val_loss', patience=25)
lr_callback = ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto'
)
def get_steps_per_epoch(n_data, batch_size):
    """ Given the data size and batch size, gives the number of steps to travers the full dataset """
    if n_data%batch_size==0:
        return int(n_data/batch_size)
    else:
        return int(n_data*1.0/batch_size)+1

In [6]:
n_epochs=50
history = vgg19_model.fit(
    train_gen, validation_data=valid_gen, 
    steps_per_epoch=get_steps_per_epoch(int(0.9*(500*200)), batch_size), 
    validation_steps=get_steps_per_epoch(int(0.1*(500*200)), batch_size),
    epochs=n_epochs, callbacks=[es_callback, lr_callback]
)
if not os.path.exists('models'):
    os.mkdir("models")
vgg19_model.save(os.path.join('models', 'VGG19_30epoch.h5'))

Epoch 1/50
1875/1875 [==============================] - 437s 230ms/step - loss: 73.8320 - accuracy: 0.1364 - val_loss: 84.5855 - val_accuracy: 0.1780 - lr: 0.0010
Epoch 2/50
1875/1875 [==============================] - 434s 231ms/step - loss: 87.3286 - accuracy: 0.2016 - val_loss: 96.6730 - val_accuracy: 0.2017 - lr: 0.0010
Epoch 3/50
1875/1875 [==============================] - 433s 231ms/step - loss: 93.7164 - accuracy: 0.2270 - val_loss: 104.8548 - val_accuracy: 0.2191 - lr: 0.0010
Epoch 4/50
1875/1875 [==============================] - 433s 231ms/step - loss: 97.4487 - accuracy: 0.2436 - val_loss: 108.5414 - val_accuracy: 0.2216 - lr: 0.0010
Epoch 5/50
1875/1875 [==============================] - 435s 232ms/step - loss: 100.7292 - accuracy: 0.2532 - val_loss: 113.6165 - val_accuracy: 0.2301 - lr: 0.0010
Epoch 6/50
1875/1875 [==============================] - ETA: 0s - loss: 102.9009 - accuracy: 0.2622
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
1875

In [7]:
# Evaluate the model
test_res = vgg19_model.evaluate(test_gen, steps=get_steps_per_epoch(500*50, batch_size))

# Print the results as a dictionary {<metric name>: <value>}
test_res_dict = dict(zip(vgg19_model.metrics_names, test_res))
print(test_res_dict)

521/521 [==============================] - 22s 43ms/step - loss: 62.8535 - accuracy: 0.3064
{'loss': 62.85350036621094, 'accuracy': 0.30640000104904175}


In [8]:
n_epochs=50
history = resnet50v2_model.fit(
    train_gen, validation_data=valid_gen, 
    steps_per_epoch=get_steps_per_epoch(int(0.9*(500*200)), batch_size), 
    validation_steps=get_steps_per_epoch(int(0.1*(500*200)), batch_size),
    epochs=n_epochs, callbacks=[es_callback, lr_callback]
)
if not os.path.exists('models'):
    os.mkdir("models")
resnet50v2_model.save(os.path.join('models', 'RESNET50_30epoch.h5'))


Epoch 1/50
1875/1875 [==============================] - 433s 230ms/step - loss: 64.6342 - accuracy: 0.0218 - val_loss: 61.0181 - val_accuracy: 0.0270 - lr: 0.0010
Epoch 2/50
1875/1875 [==============================] - 430s 230ms/step - loss: 62.2299 - accuracy: 0.0304 - val_loss: 58.3081 - val_accuracy: 0.0319 - lr: 0.0010
Epoch 3/50
1875/1875 [==============================] - 432s 230ms/step - loss: 61.2347 - accuracy: 0.0357 - val_loss: 58.4649 - val_accuracy: 0.0446 - lr: 0.0010
Epoch 4/50
1875/1875 [==============================] - 435s 232ms/step - loss: 60.4660 - accuracy: 0.0382 - val_loss: 55.1613 - val_accuracy: 0.0398 - lr: 0.0010
Epoch 5/50
1875/1875 [==============================] - 430s 229ms/step - loss: 60.5263 - accuracy: 0.0407 - val_loss: 70.9044 - val_accuracy: 0.0347 - lr: 0.0010
Epoch 6/50
1875/1875 [==============================] - 434s 231ms/step - loss: 60.1795 - accuracy: 0.0417 - val_loss: 56.4137 - val_accuracy: 0.0432 - lr: 0.0010
Epoch 7/50
1875/1875 [

In [9]:
# Evaluate the model
test_res = resnet50v2_model.evaluate(test_gen, steps=get_steps_per_epoch(500*50, batch_size))

# Print the results as a dictionary {<metric name>: <value>}
test_res_dict = dict(zip(resnet50v2_model.metrics_names, test_res))
print(test_res_dict)

521/521 [==============================] - 14s 27ms/step - loss: 9.1434 - accuracy: 0.1128
{'loss': 9.143423080444336, 'accuracy': 0.1128000020980835}


In [10]:
n_epochs=50
history = inceptionv3_model.fit(
    train_gen_inceptionV3, validation_data=valid_gen_inceptionV3, 
    steps_per_epoch=get_steps_per_epoch(int(0.9*(500*200)), batch_size), 
    validation_steps=get_steps_per_epoch(int(0.1*(500*200)), batch_size),
    epochs=n_epochs, callbacks=[es_callback, lr_callback])
if not os.path.exists('models'):
    os.mkdir("models")
inceptionv3_model.save(os.path.join('models', 'InceptionV3_30epoch.h5'))

Epoch 1/50
1875/1875 [==============================] - 777s 413ms/step - loss: 20.8447 - accuracy: 0.0173 - val_loss: 21.2633 - val_accuracy: 0.0210 - lr: 0.0010
Epoch 2/50
1875/1875 [==============================] - 777s 414ms/step - loss: 18.9357 - accuracy: 0.0226 - val_loss: 17.5337 - val_accuracy: 0.0211 - lr: 0.0010
Epoch 3/50
1875/1875 [==============================] - 776s 414ms/step - loss: 18.5674 - accuracy: 0.0267 - val_loss: 19.3466 - val_accuracy: 0.0246 - lr: 0.0010
Epoch 4/50
1875/1875 [==============================] - 778s 415ms/step - loss: 18.4079 - accuracy: 0.0285 - val_loss: 16.4905 - val_accuracy: 0.0307 - lr: 0.0010
Epoch 5/50
1875/1875 [==============================] - 783s 418ms/step - loss: 18.4307 - accuracy: 0.0283 - val_loss: 19.0468 - val_accuracy: 0.0336 - lr: 0.0010
Epoch 6/50
1875/1875 [==============================] - 779s 416ms/step - loss: 18.0618 - accuracy: 0.0293 - val_loss: 17.8902 - val_accuracy: 0.0301 - lr: 0.0010
Epoch 7/50
1875/1875 [

ResourceExhaustedError: Graph execution error:

2 root error(s) found.
  (0) RESOURCE_EXHAUSTED:  MemoryError: Unable to allocate 1.02 MiB for an array with shape (299, 299, 3) and data type float32
Traceback (most recent call last):

  File "C:\Users\docto\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__

  File "C:\Users\docto\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper

  File "C:\Users\docto\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1035, in generator_py_func

  File "C:\Users\docto\anaconda3\envs\py310\lib\site-packages\keras\engine\data_adapter.py", line 903, in wrapped_generator
    for data in generator_fn():

  File "C:\Users\docto\anaconda3\envs\py310\lib\site-packages\keras\engine\data_adapter.py", line 1050, in generator_fn
    yield x[i]

  File "C:\Users\docto\anaconda3\envs\py310\lib\site-package

In [ ]:
#Run out of resources for last run, but as you can see, highest accuracy from last epoch 37 is 0.091


In [7]:
# # Evaluate the model
# test_res = inceptionv3_model.evaluate(test_gen_inceptionV3, steps=get_steps_per_epoch(500*50, batch_size))

# # Print the results as a dictionary {<metric name>: <value>}
# test_res_dict = dict(zip(inceptionv3_model.metrics_names, test_res))
# print(test_res_dict)


# can't run this part saddly, run out of resources during training and not enough time left
# luckly it's only inceptionV3, so it's just something extra I was trying to do

In [9]:
from tensorflow.keras.applications import VGG16,ResNet101V2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D


def create_ResNet101V2_model(input_shape=(224, 224, 3), num_classes=200):
    base_model = ResNet101V2(weights='imagenet', include_top=False, input_shape=input_shape)
    x = GlobalAveragePooling2D()(base_model.output)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model

resnet101V2_model = create_ResNet101V2_model()
resnet101V2_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
def create_vgg16_model(input_shape=(224, 224, 3), num_classes=200):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    x = Flatten()(base_model.output)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model

vgg16_model = create_vgg16_model()
vgg16_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#vgg16_model.summary()
#resnet101V2_model.summary()

58889256/58889256 [==============================] - 2s 0us/step


In [16]:
n_epochs=15
history = vgg16_model.fit(
    train_gen, validation_data=valid_gen, 
    steps_per_epoch=get_steps_per_epoch(int(0.9*(500*200)), batch_size), 
    validation_steps=get_steps_per_epoch(int(0.1*(500*200)), batch_size),
    epochs=n_epochs, callbacks=[es_callback, lr_callback]
)
if not os.path.exists('models'):
    os.mkdir("models")
vgg16_model.save(os.path.join('models', 'vgg16_model.h5'))


Epoch 1/15
1875/1875 [==============================] - 427s 227ms/step - loss: 78.4746 - accuracy: 0.3447 - val_loss: 96.5718 - val_accuracy: 0.2804 - lr: 1.0000e-04
Epoch 2/15
1875/1875 [==============================] - 430s 229ms/step - loss: 75.3584 - accuracy: 0.3487 - val_loss: 94.3162 - val_accuracy: 0.2927 - lr: 1.0000e-04
Epoch 3/15
1875/1875 [==============================] - 427s 228ms/step - loss: 72.5291 - accuracy: 0.3503 - val_loss: 91.0569 - val_accuracy: 0.2901 - lr: 1.0000e-04
Epoch 4/15
1875/1875 [==============================] - 428s 228ms/step - loss: 69.6988 - accuracy: 0.3573 - val_loss: 89.8522 - val_accuracy: 0.2858 - lr: 1.0000e-04
Epoch 5/15
1875/1875 [==============================] - 429s 229ms/step - loss: 68.4815 - accuracy: 0.3567 - val_loss: 86.6571 - val_accuracy: 0.2898 - lr: 1.0000e-04
Epoch 6/15
1875/1875 [==============================] - 430s 229ms/step - loss: 66.9124 - accuracy: 0.3592 - val_loss: 85.6055 - val_accuracy: 0.2947 - lr: 1.0000e-0

In [17]:
# Evaluate the model
test_res = vgg16_model.evaluate(test_gen, steps=get_steps_per_epoch(500*50, batch_size))

# Print the results as a dictionary {<metric name>: <value>}
test_res_dict = dict(zip(vgg16_model.metrics_names, test_res))
print(test_res_dict)

521/521 [==============================] - 18s 34ms/step - loss: 89.4672 - accuracy: 0.3041
{'loss': 89.46724700927734, 'accuracy': 0.30410000681877136}


In [18]:
n_epochs=15
history = resnet101V2_model.fit(
    train_gen, validation_data=valid_gen, 
    steps_per_epoch=get_steps_per_epoch(int(0.9*(500*200)), batch_size), 
    validation_steps=get_steps_per_epoch(int(0.1*(500*200)), batch_size),
    epochs=n_epochs, callbacks=[es_callback, lr_callback]
)
if not os.path.exists('models'):
    os.mkdir("models")
resnet101V2_model.save(os.path.join('models', 'resnet101V2_model.h5'))


Epoch 1/15
1875/1875 [==============================] - 438s 231ms/step - loss: 164.6752 - accuracy: 0.0154 - val_loss: 161.7328 - val_accuracy: 0.0209 - lr: 0.0010
Epoch 2/15
1875/1875 [==============================] - 435s 232ms/step - loss: 165.1906 - accuracy: 0.0213 - val_loss: 153.2496 - val_accuracy: 0.0193 - lr: 0.0010
Epoch 3/15
1875/1875 [==============================] - 437s 233ms/step - loss: 164.0427 - accuracy: 0.0245 - val_loss: 156.5984 - val_accuracy: 0.0268 - lr: 0.0010
Epoch 4/15
1875/1875 [==============================] - 436s 233ms/step - loss: 161.8978 - accuracy: 0.0263 - val_loss: 174.6801 - val_accuracy: 0.0319 - lr: 0.0010
Epoch 5/15
1875/1875 [==============================] - 437s 233ms/step - loss: 164.6766 - accuracy: 0.0274 - val_loss: 154.1006 - val_accuracy: 0.0281 - lr: 0.0010
Epoch 6/15
1875/1875 [==============================] - 439s 234ms/step - loss: 163.6740 - accuracy: 0.0281 - val_loss: 149.5718 - val_accuracy: 0.0339 - lr: 0.0010
Epoch 7/15

In [19]:
# Evaluate the model
test_res = resnet101V2_model.evaluate(test_gen, steps=get_steps_per_epoch(500*50, batch_size))

# Print the results as a dictionary {<metric name>: <value>}
test_res_dict = dict(zip(resnet101V2_model.metrics_names, test_res))
print(test_res_dict)

521/521 [==============================] - 27s 51ms/step - loss: 154.1031 - accuracy: 0.0356
{'loss': 154.10311889648438, 'accuracy': 0.035599999129772186}


In [ ]:
#Here's inceptionV4
# please refer to the JDong_HW5_InceptionV4 notebook for results
# I had to use pytorch for it and not tensorflow

In [ ]:
#models are too large to upload to black board and thus kept local. 

In [21]:
# for analysis propurses
print("VGG16")
vgg16_model.summary()
print("VGG19")
vgg19_model.summary()
print("Resnet50")
resnet50v2_model.summary()
print("Resnet100")
resnet101V2_model.summary()
print("InceptionV3")
inceptionv3_model.summary()

VGG16
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)     